In [15]:
#change ovito export xyz file to RBS simulation input data file
#ovito export xyz file format is :
#25200000
#Lattice="0.0 0.0 5824.86343385 0.0 211.4 0.0 215.0 0.0 0.0" Origin="-86.0 -90.6 -2900.7819900573" Properties=pos:R:3:species:S:1:id:I:1
#-79.5354212843 -89.9262116686 -2899.4576111988 Ga 1
#-80.8037735621 -88.2192198017 -2890.5819782791 Ga 2
#-85.4477714422 -86.5041908662 -2899.4191513497 Ga 3
#RBS data file example:
#25200000
#Lattice="215.00000 0.00000 0.00000 0.00000 214.4 0.00000 0.00000 0.00000 5824.86343385"
#Ga -79.5354212843 -89.9262116686 -2899.4576111988  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1


In [3]:
import pandas as pd
import numpy as np
import os
import re

In [22]:

# Replace 'your_file.txt' with the actual path to your file
file_path = '/home/heruhe/Desktop/Ga2o3/RBS/gamma110/gamma110.in'
file_path='/home/heruhe/Desktop/Ga2o3/combine_cells/2680PKA'
try:
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i < 3:
                print("Line {}: {}".format(i + 1, line.strip())) # strip() removes leading/trailing whitespace
                
            else:
                break
except FileNotFoundError:
    print("File not found:", file_path)
except Exception as e:
    print("An error occurred:", str(e))

Line 1: 3850240
Line 2: Lattice="100.5915804555 0.0 0.0 0.0 0.0 4495.5031814909 0.0 94.7643495008 0.0" Origin="0.0 0.0 -2199.9270888147" Properties=pos:R:3:species:S:1:id:I:1
Line 3: 70.9942148347 3.5376494221 -2177.7914012973 O1 1


In [23]:
df=pd.read_csv(file_path,skiprows=2,header=None,sep='\s+')

In [24]:
df.columns=['x','y','z','type','ID']


In [26]:

# Function to convert OVITO XYZ format to RBS format for a single line
#Move the cell origin to zero
def convert_xyz_to_rbs(line,bx,by,bz):
    parts = line.split()
    if len(parts) == 5:
        x, y, z, species, id_val = parts
        x=float(x)-bx
        y=float(y)-by
        z=float(z)-bz
        if species=='Ga1':
            id_val=1
        if species=='Ga2':
            id_val=2
        if species=='O1':
            id_val=3
        if species=='O2':
            id_val=4
        if species=='O3':
            id_val=5

        return f"{species} {x} {y} {z} {id_val}\n"
    else:
        return None
    
    



# Replace 'input.xyz' with your input XYZ file and 'output.rbs' with your desired output RBS file
input_file = file_path
output_file = '/home/heruhe/Desktop/Ga2o3/combine_cells/2680PKA.in'

try:
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Copy the first two lines as is (header lines)
        for i, line in enumerate(infile):
            # Copy the first two lines as is (header lines)
            if i ==0:
                #print(line)
                outfile.write(line)
            if i==1:
                # Define a regular expression pattern to match 'Lattice="..."' and capture the content inside the quotes
                pattern = r'Lattice="([^"]*)"'
                # Define a regular expression pattern to match 'Origin="..."' and capture the content inside the quotes
                pattern2 = r'Origin="([^"]*)"'

                # Use re.search() to find the pattern in the input string
                match = re.search(pattern, line)
                match2 = re.search(pattern2, line)
                
                # Check if a match was found
                if match and match2:
                    lattice_value =match.group(1) 
                    numbers = [float(x) for x in lattice_value.split()]
                    origin_value =match2.group(1) 
                    numbers2 = [float(x) for x in origin_value.split()]
                    print(numbers2)
                    # Create a 3x3 matrix from the list of numbers
                    matrix = np.array(numbers).reshape(3, 3)

                    # Print the resulting matrix
                    print(matrix)
                    y=matrix[2][1]
                    z=matrix[1][2]
                    x=matrix[0][0]
                    move_z=numbers2[2]
                    move_y=numbers2[1]
                    move_x=numbers2[0]
                    
                    Lattice='Lattice="'+str(x)+' 0.00000'+' 0.00000'+' 0.00000 '+str(y)+' 0.00000'+' 0.00000'+' 0.00000 '+str(z)+'"\n'
                    outfile.write(Lattice)
                else:
                    print("Lattice not found in the input string.")
            if i>1:
                convert_line=convert_xyz_to_rbs(line,move_x,move_y,move_z)
                outfile.write(convert_line)
                
except FileNotFoundError:
    print("File not found:", input_file)
except Exception as e:
    print("An error occurred:", str(e))

print("Conversion complete.")

[0.0, 0.0, -2199.9270888147]
[[ 100.59158046    0.            0.        ]
 [   0.            0.         4495.50318149]
 [   0.           94.7643495     0.        ]]
Conversion complete.
